In [1]:
import numpy as np
import shap
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:

def our_generator():
    for i in range(2):
        x = np.random.rand(28,28)
        y = np.random.randint(1,10, size=1)
        yield x,y
dataset = tf.data.Dataset.from_generator(our_generator, (tf.float32, tf.int16))
iter = dataset.make_initializable_iterator()
el = iter.get_next()
with tf.Session() as sess:
    sess.run(iter.initializer)
    print(sess.run(el))

W0402 09:23:48.825565 139701041714944 deprecation.py:323] From <ipython-input-2-1316617d0142>:8: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


(array([[9.23568666e-01, 7.15557456e-01, 6.42988384e-02, 3.28886122e-01,
        8.24291646e-01, 3.07671398e-01, 9.82494354e-01, 8.27024877e-01,
        3.51636797e-01, 9.49251175e-01, 5.62024593e-01, 6.28910899e-01,
        4.85443503e-01, 1.74105719e-01, 8.66397500e-01, 2.09385782e-01,
        4.82944101e-01, 7.82969236e-01, 7.26818085e-01, 4.15929675e-01,
        6.91914260e-01, 3.05846989e-01, 9.45499539e-02, 9.47320938e-01,
        4.68461424e-01, 1.42989591e-01, 1.20360687e-01, 1.36567503e-01],
       [1.79281458e-01, 6.41529799e-01, 2.76150316e-01, 1.27886578e-01,
        9.11585212e-01, 3.01052898e-01, 9.28364694e-01, 3.37106407e-01,
        7.77626932e-01, 8.38802934e-01, 5.76275051e-01, 4.66295660e-01,
        6.93947077e-01, 7.51414239e-01, 5.00512958e-01, 2.93324322e-01,
        9.91494298e-01, 9.86662745e-01, 4.82010454e-01, 4.49926138e-01,
        5.69170713e-01, 8.56479228e-01, 3.03642571e-01, 7.29480803e-01,
        8.44889432e-02, 6.48748398e-01, 2.65788168e-01, 7.0906

In [69]:
path = "/home/advice/Python/SR/Data/kdd/uci/uci_creditcard-train-0.2-0.0.csv"
data = pd.read_csv(path).sample(505)
data = data.fillna(1)
#data.dropna( inplace=True)

In [70]:
data.columns.tolist()

['ID',
 'LIMIT_BAL',
 'SEX',
 'EDUCATION',
 'MARRIAGE',
 'AGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6',
 'default payment next month',
 'sep_idx']

In [71]:
# load a csv
total_var = ['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default payment next month', 'sep_idx']
in_var = ["LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6",
          "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6",
          "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"]
target_var = ['default payment next month']
in_var + target_var

['LIMIT_BAL',
 'SEX',
 'EDUCATION',
 'MARRIAGE',
 'AGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6',
 'default payment next month']

In [72]:
dataset = tf.contrib.data.make_csv_dataset(path, 
                                           column_names= total_var , 
                                           select_columns = in_var + target_var,
                                           label_name= target_var[0] , 
                                           batch_size=32)
def pack_features_vector(features, labels):
    """피처를 하나의 배열로 패킹합니다."""
    features = tf.stack(list(features.values()), axis=1)
    print(features)
    return features, labels
train_dataset = dataset.map(pack_features_vector)
iter = train_dataset.make_one_shot_iterator()
features , labels = iter.get_next()

Tensor("stack:0", shape=(32, 23), dtype=float32)


In [73]:
with  tf.Session() as sess:
    print(sess.run(features).shape)
# inputs, labels = next['text'], next['sentiment']
# with  tf.Session() as sess:
#     sess.run([inputs, labels])

(32, 23)


In [74]:
target = "default payment next month"
label = data.pop(target)

In [75]:
labels = pd.get_dummies(label)

In [76]:
X = tf.placeholder(tf.float32 , shape = [None,data.shape[1]])
y = tf.placeholder(tf.float32 , shape = [None,labels.shape[1]])

In [77]:
def _function(feature, label) :
    """전처리 함수"""
    print(feature)
    return feature , label

In [78]:
data_tuple = (X,y)
dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
#dataset.map(_function)
dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.batch(batch_size=10, drop_remainder=True)
EPOCHS = 10
#dataset.repeat(count=EPOCHS)

In [79]:
iter = dataset.make_initializable_iterator()
feature_batch , label_batch = iter.get_next()

In [80]:
iter2 = dataset.make_initializable_iterator()
feature_batch_2 , label_batch_2 = iter2.get_next()

In [81]:
with tf.Session() as sess:
    for i in range(EPOCHS):
        print(i)
        sess.run(iter.initializer , feed_dict= {X : data.values,
                                            y : labels.values
                                           }) # switch to train dataset
        sess.run(iter2.initializer , feed_dict= {X : data.values,
                                                y : labels.values
                                               }) # switch to train dataset
        while True :
            try :
                sess.run(feature_batch)
#                 print(sess.run(feature_batch).shape)
#                 print(sess.run(feature_batch_2).shape)
            except tf.errors.OutOfRangeError :
                break

0
1
2
3
4
5
6
7
8
9


# tf.py_func 적용해보기

* Tensorflow 중간 연산에 Python 연산을 사용하고 싶을 때 사용하는 것
* 하고자 하는 시나리오
    * 전처리 Pipeline 학습시킴
    * csv로 배치 단위로 읽어오기
    * 그다음에 바로 적용해서 처리하기
* 단점
    * oversampling 같은 기법들을 사용하지 못함
    * 전체적인 추론을 하기 위한 구조가 따로 필요함

## https://docs.w3cub.com/tensorflow~python/tf/py_func/

## https://github.com/hccho2/hccho2FirstGitProject/blob/master/tensorflowTest.py

## https://github.com/cesc-park/attend2u/issues/2

In [82]:
X = tf.placeholder(tf.float32 , shape = [None,data.shape[1]])
y = tf.placeholder(tf.float32 , shape = [None,labels.shape[1]])

def _function(feature, label) :
    """전처리 함수"""
    print(feature.shape)
    return feature , label

data_tuple = (X,y)
dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
# dataset.map(lambda xx, yy : tf.py_func(_function, [])
dataset = dataset.shuffle(buffer_size=100)
dataset = dataset.batch(batch_size=10, drop_remainder=True)
EPOCHS = 10
#dataset.repeat(count=EPOCHS)

iter = dataset.make_initializable_iterator()
feature_batch , label_batch = iter.get_next()

iter2 = dataset.make_initializable_iterator()
feature_batch_2 , label_batch_2 = iter2.get_next()

In [83]:
def my_func(x:np.ndarray) :
    print(x.shape)
    ## .astype(np.float32) 중요
    x2 = x + np.random.normal(loc = 100.0 , scale = 1.0 , size = x.shape).astype(np.float32)
    print(x2)
    return x2

new_x = tf.py_func(my_func , [feature_batch] , tf.float32)
new_x2 = tf.reshape(new_x, list(feature_batch.get_shape()))
new_x2

<tf.Tensor 'Reshape_9:0' shape=(10, 25) dtype=float32>

In [86]:
w = tf.get_variable("aa",shape=[25,10])
trans_x = tf.matmul(new_x2 , w)

In [84]:
import numpy as np

In [93]:
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init)
    for i in range(EPOCHS):
        print(i)
        sess.run(iter.initializer , feed_dict= {X : data.values,
                                            y : labels.values
                                           }) # switch to train dataset
        sess.run(iter2.initializer , feed_dict= {X : data.values,
                                                y : labels.values
                                               }) # switch to train dataset
        while True :
            try :
                print(sess.run(trans_x))
                #print(sess.run(trans_x))
#                 print(sess.run(feature_batch).shape)
#                 print(sess.run(feature_batch_2).shape)
            except tf.errors.OutOfRangeError :
                break

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:1 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: TITAN X (Pascal), pci bus id: 0000:03:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: TITAN X (Pascal), pci bus id: 0000:04:00.0, compute capability: 6.1

0
(10, 25)
[[2.97154219e+04 2.00100938e+05 1.02051781e+02 1.00285484e+02
  1.00446014e+02 1.00571602e+02 1.01558212e+02 1.02265129e+02
  9.97932892e+01 1.01639694e+02 1.00983322e+02 1.00707352e+02
  1.00299095e+02 1.92660312e+05 1.00313805e+02 1.94880156e+05
  1.00688774e+02 1.93367078e+05 7.10485059e+03 1.00743073e+02
  7.16983154e+03 6.61350488e+03 7.18172168e+03 1.02210400e+04
  1.00968552e+02]
 [4.21742480e+03 2.30099828e+05 1.02648346e+02 1.01742798e+02
  1.00418739e+02 1.00994728e+

[[ 1.5667066e+05  7.4473641e+04 -9.1503633e+04  4.0792734e+04
   1.8204006e+05 -8.2983242e+04  8.4091812e+04  4.2720453e+04
  -8.4611367e+04 -2.5219264e+04]
 [ 1.2701175e+04  8.0592285e+03 -2.6245642e+02 -1.2549316e+04
   3.5807358e+03 -5.5423271e+03 -7.5440703e+03 -8.0300503e+03
  -1.0830496e+04 -8.4739111e+03]
 [ 2.6813980e+04  1.6988975e+04 -6.8217178e+03 -6.3834766e+02
   2.5462125e+04  2.2085547e+03  1.0796375e+03 -7.3041826e+03
  -2.3904422e+04  1.0335969e+04]
 [ 6.3579250e+04  2.6090576e+04 -2.1663967e+04 -1.2037134e+03
   4.9119043e+04 -2.0759727e+04  2.0215092e+04  6.5860054e+03
  -3.4268266e+04 -1.1537009e+04]
 [ 8.4161578e+04  6.7356867e+04 -2.3302982e+04  1.2962750e+04
   9.3296500e+04  3.3496914e+04  2.8127027e+04 -5.4708496e+03
  -1.1825297e+05  7.7040250e+04]
 [ 3.0803850e+04  1.2830622e+04 -1.3917228e+04  5.1873848e+03
   3.0790648e+04 -1.2412097e+04  1.4466109e+04  6.5002744e+03
  -1.7556834e+04 -5.4075903e+03]
 [ 3.2101162e+05  2.4614766e+05  4.5377676e+03 -7.3442984e

[[  19071.098     9748.117     6187.1646  -13739.632    -1746.5093
     9750.13     -6711.3794   -9786.966   -27467.84      4217.289 ]
 [  36783.875    19256.87      2952.38     -3551.609    12388.113
     7847.8496    7425.4062    5242.695   -27094.824    11663.656 ]
 [  36306.105    14446.619   -10029.887     -545.999    28546.986
   -10319.304    12041.842     2700.7551  -20546.686    -4171.9023]
 [  92543.7      68108.18    -38867.273    -9399.963   130726.26
   -53776.652    47148.223     -842.3125  -99410.61    -15797.778 ]
 [  25686.844     6999.3896   -6527.463    -2546.1985   16900.94
    -3466.5781    3283.9705     428.2827  -15512.895    -5694.0366]
 [  97496.73     41423.848   -50927.445    18707.564   101028.84
   -44665.598    41790.22     18243.252   -52208.84    -15084.08  ]
 [  83789.445    49169.14    -24403.121    -4269.302    60664.938
   -39791.316    36025.945     4161.63    -37624.684    -8264.471 ]
 [ 174680.08     85075.38    -74173.41     49090.78    194962.39

(10, 25)
[[1.78659473e+04 3.01012520e+04 1.03023438e+02 1.03256126e+02
  1.00235741e+02 1.56083969e+02 1.00984238e+02 1.02005295e+02
  1.02387383e+02 1.00831253e+02 1.00508484e+02 1.04182175e+02
  2.52583301e+04 2.45648691e+04 2.41887266e+04 2.59906582e+04
  2.52757715e+04 2.69701133e+04 9.92550812e+01 4.00132080e+02
  2.60016064e+03 9.92243042e+01 2.19959497e+03 9.90121841e+01
  1.02361534e+02]
 [1.54393828e+04 1.50099391e+05 1.01286163e+02 1.01591721e+02
  1.01385170e+02 9.90678177e+01 1.02524094e+02 1.00932762e+02
  9.88231354e+01 1.00552071e+02 9.96540527e+01 1.00609093e+02
  9.99615173e+01 1.33782141e+05 1.01438522e+02 1.47228859e+05
  1.02068687e+02 7.20772656e+04 7.09977051e+03 1.00574074e+02
  3.09993115e+03 5.34330420e+03 1.00988876e+02 4.10028125e+03
  9.94616165e+01]
 [2.04877910e+04 1.00991387e+04 9.97290268e+01 1.01185181e+02
  1.02168442e+02 1.22326141e+02 1.00433128e+02 1.01398865e+02
  9.90359116e+01 9.86865005e+01 9.89621887e+01 9.85143814e+01
  8.79153259e+02 1.037609

[[ 2.01630713e+03  2.60098734e+05  1.01685646e+02  1.01171852e+02
   1.00169762e+02  1.32366455e+02  9.91406860e+01  9.89637680e+01
   9.81766129e+01  9.98478317e+01  1.00421494e+02  1.01212860e+02
   1.01180054e+02  3.06394745e+02 -7.10513855e+02  2.28977319e+03
   1.01681343e+02  1.00078148e+02  1.02191109e+02  1.00008827e+02
   3.09978223e+03  9.80310593e+01  9.96743240e+01  9.96447983e+01
   1.02180908e+02]
 [ 2.08974121e+04  7.00997656e+04  1.01917175e+02  1.02358002e+02
   1.01396622e+02  1.48554230e+02  9.92233505e+01  1.01236702e+02
   1.00788239e+02  1.01279305e+02  1.00685768e+02  1.01962273e+02
   5.92547656e+04  5.45719727e+04  5.13584297e+04  4.86640391e+04
   4.76112539e+04  1.01005508e+02  2.12069141e+03  2.11140137e+03
   2.11409326e+03  1.01482719e+02  2.09964233e+03  2.30144360e+03
   1.01580261e+02]
 [ 7.87323291e+03  5.01002656e+04  1.01916359e+02  1.01371841e+02
   1.01987442e+02  1.40503967e+02  1.00651283e+02  1.00957344e+02
   1.01063248e+02  9.95958939e+01  1.0

...

8
(10, 25)
[[2.57617734e+04 1.00098172e+05 1.02502838e+02 1.02009567e+02
  1.01550453e+02 1.34368408e+02 1.00593910e+02 9.92073135e+01
  1.00863670e+02 1.00927910e+02 9.86166000e+01 1.00268265e+02
  1.00938583e+02 1.02080850e+04 8.12274805e+03 9.83477051e+03
  1.73833184e+04 8.63894922e+03 2.10978052e+03 1.24770361e+03
  3.19350122e+03 1.10243127e+03 1.10376782e+03 5.06875366e+02
  1.00051010e+02]
 [2.18981680e+04 2.00099672e+05 1.02092224e+02 1.02081718e+02
  1.00487099e+02 9.93540802e+01 9.86554947e+01 9.86189651e+01
  9.89941940e+01 9.86938782e+01 9.86082993e+01 9.93488922e+01
  2.39775122e+03 1.59124573e+03 1.76838049e+03 2.40538879e+02
  1.20034131e+03 6.96207764e+03 1.58896289e+03 1.76778711e+03
  2.43923309e+02 1.20024585e+03 6.96098389e+03 9.94386444e+01
  1.00140663e+02]
 [1.10606553e+04 1.20099234e+05 9.99081116e+01 1.01508614e+02
  1.01883904e+02 1.27808250e+02 1.00217064e+02 9.98831940e+01
  9.95587845e+01 9.94664459e+01 1.00515175e+02 9.99514160e+01
  9.69327031e+04 7.9019

[[ 5.56159766e+03  2.30100094e+05  1.01533241e+02  1.03420952e+02
   1.01124779e+02  1.38493835e+02  9.93867035e+01  1.00792343e+02
   1.00783234e+02  1.00598488e+02  9.89463730e+01  9.93080597e+01
   1.00100418e+02 -5.10779343e+01  6.16924011e+02  1.53744522e+02
   7.02778687e+02  1.00984360e+02  1.00523407e+02  7.67982422e+02
   1.01500084e+02  7.99481323e+02  1.02993691e+04  9.97674026e+01
   9.96686859e+01]
 [ 7.42899268e+03  1.80097219e+05  1.01828583e+02  1.00666122e+02
   1.01980072e+02  1.26869232e+02  9.79871750e+01  9.95995636e+01
   9.78125992e+01  1.00645462e+02  1.02339279e+02  9.94319687e+01
   7.41112061e+02  1.91299622e+03  5.40720703e+03  2.81037227e+04
   1.05687695e+03  1.01715530e+02  1.91246985e+03  5.44825684e+03
   2.81033145e+04  1.05865784e+03  9.93225403e+01  1.01254326e+02
   1.02721733e+02]
 [ 7.20280322e+03  5.00995195e+04  1.01014198e+02  1.03381599e+02
   1.01616043e+02  1.35411102e+02  9.86069412e+01  9.97432938e+01
   1.00001877e+02  1.00557480e+02  9.8

[[ 9.01524609e+04  4.47258398e+04 -5.20122578e+04  2.34786465e+04
   1.06993773e+05 -4.82101094e+04  5.11736484e+04  2.49225645e+04
  -4.94013320e+04 -1.29921631e+04]
 [ 1.03403484e+05  5.30133672e+04 -4.47487344e+04  7.56292773e+03
   9.89795312e+04 -4.61248320e+04  5.56588008e+04  1.76093125e+04
  -6.18510391e+04 -8.09089941e+03]
 [ 1.50381875e+04  1.50860596e+01  6.73560693e+03  1.17850293e+04
   1.03727930e+04  6.93264648e+03  1.30046016e+04  9.02721741e+02
  -2.49804277e+04  3.06670605e+03]
 [ 7.13726953e+04  3.78680195e+04 -3.77940742e+04  1.56742891e+04
   8.02701094e+04 -3.72919180e+04  4.15090391e+04  1.98629141e+04
  -3.71985078e+04 -8.33357031e+03]
 [ 1.98375938e+04  1.75614648e+04  6.12842383e+03 -1.63867988e+04
   1.24951562e+03  2.02664551e+03 -2.04155098e+04 -1.10695938e+04
  -2.25619922e+04 -1.15492517e+03]
 [ 1.06585812e+05  5.04275078e+04  2.27324805e+04 -2.61951797e+04
   1.65387363e+04  4.46859609e+04  7.00532891e+04  3.10600312e+04
  -8.53607266e+04  3.27674160e+04

[[  21289.44     11580.097    16071.165     2936.6602    2170.5771
    14034.332     2617.8945   -2698.4592  -42660.53      9999.605 ]
 [ 146091.98    103857.516   -36556.996    -5693.8203  126357.34
   -61852.844     6078.336     4502.211  -101517.484   -21761.438 ]
 [  88354.07     89035.98     32405.266    14630.691   112080.64
    14526.629    60204.508  -132197.88   -334319.94     63445.086 ]
 [  60937.227    43187.977    10129.075   -48732.598    -8781.291
   -16179.05    -39663.195     5098.269   -26574.21    -29306.352 ]
 [  23635.137    16732.338      505.2649  -15574.539    10948.924
     2836.7427   -8688.035   -12441.421   -23727.008     1700.1427]
 [  55005.21     39276.98     25327.896   -13888.83      1117.9258
    13119.205    -5759.9727   -2424.1294  -81187.06     10446.086 ]
 [  42026.473    19174.777   -24714.703     8790.551    45536.69
   -22862.818    20840.219    10105.45    -24868.385    -5902.5493]
 [  72095.29     29879.582    -5407.6855   22857.633    64231.3

[[1.53622568e+04 2.90101000e+05 1.03410851e+02 1.01175858e+02
  1.02204269e+02 1.28345200e+02 9.96651230e+01 9.95250015e+01
  9.90509033e+01 1.02052338e+02 1.00668488e+02 9.98527298e+01
  1.94136875e+04 1.01800949e+02 9.96635437e+01 3.15046914e+04
  3.11053164e+04 2.92241074e+04 1.50999639e+04 1.01872208e+02
  3.09810962e+03 1.01087387e+02 9.90252533e+01 4.37991016e+03
  1.01156158e+02]
 [1.89081875e+04 5.01004102e+04 1.01119919e+02 1.00716362e+02
  1.02684563e+02 1.00897514e+02 1.01314819e+02 9.93730011e+01
  1.00215706e+02 1.00925484e+02 1.00221390e+02 9.97354507e+01
  3.13053457e+04 1.02181175e+02 2.45175898e+04 2.60388047e+04
  1.00864525e+02 2.82916699e+04 2.09990234e+03 2.10004004e+03
  2.09883130e+03 1.60013867e+03 1.60189343e+03 1.02632141e+02
  1.01298172e+02]
 [1.50367529e+04 8.01010000e+04 1.00546425e+02 1.01810143e+02
  1.01032829e+02 1.41524841e+02 9.96320724e+01 1.00707550e+02
  9.97216415e+01 1.00103065e+02 9.89765778e+01 1.00368645e+02
  7.84366797e+04 7.80608047e+04 7.